In [3]:
import pandas as pd
import numpy as np


# Cargar los datos
df = pd.read_csv("G:\Mi unidad\Data Science\proyecto\Proyecto-predictor-de-precios-de-propiedades-en-EEUU\src/EDA_Final.csv")
df


,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,one_adult_no_kids_living_wage,...,total_incidents,total_killed,total_injured,population_2020,crime_index,house_size_acre,city_n,state_n,log_price,log_house_size
0,180000.0,2.0,1.0,0.340000,732726.0,Agawam,Massachusetts,1001.0,676.0,19.170,...,5981.0,472.0,1701.0,7029917.0,0.000851,0.015519,0,0,12.100712,6.516193
1,169900.0,2.0,2.0,0.020478,1194249.0,Agawam,Massachusetts,1001.0,892.0,19.170,...,5981.0,472.0,1701.0,7029917.0,0.000851,0.020478,0,0,12.042965,6.793466
2,242000.0,2.0,2.0,0.032782,913299.0,Agawam,Massachusetts,1001.0,1428.0,19.170,...,5981.0,472.0,1701.0,7029917.0,0.000851,0.032782,0,0,12.396693,7.264030
3,299950.0,2.0,2.0,0.038085,560791.0,Agawam,Massachusetts,1001.0,1659.0,19.170,...,5981.0,472.0,1701.0,7029917.0,0.000851,0.038085,0,0,12.611371,7.413970
4,239900.0,3.0,1.0,0.460000,849166.0,Agawam,Massachusetts,1001.0,1196.0,19.170,...,5981.0,472.0,1701.0,7029917.0,0.000851,0.027456,0,0,12.387977,7.086738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691623,359900.0,4.0,2.0,0.330000,353094.0,Richland,Washington,99354.0,3600.0,15.625,...,3434.0,853.0,1287.0,7705281.0,0.000446,0.082645,2026,40,12.793581,8.188689
1691624,350000.0,3.0,2.0,0.100000,1062149.0,Richland,Washington,99354.0,1616.0,15.625,...,3434.0,853.0,1287.0,7705281.0,0.000446,0.037098,2026,40,12.765688,7.387709
1691625,440000.0,6.0,3.0,0.500000,405677.0,Richland,Washington,99354.0,3200.0,15.625,...,3434.0,853.0,1287.0,7705281.0,0.000446,0.073462,2026,40,12.994530,8.070906
1691626,179900.0,2.0,1.0,0.090000,761379.0,Richland,Washington,99354.0,933.0,15.625,...,3434.0,853.0,1287.0,7705281.0,0.000446,0.021419,2026,40,12.100156,6.838405


In [5]:
import pandas as pd


# Crear un diccionario de estados a ciudades únicas
data_dict = {}

for estado, ciudades in df.groupby('state')['city']:
    data_dict[estado] = list(set(ciudades))

print(data_dict)


{'Alabama': ['Brilliant', 'Mulga', 'Dadeville', 'Brierfield', 'Luverne', 'Fruithurst', 'West Jefferson', 'Boligee', 'Anderson', 'Stapleton', 'Greensboro', 'Blountsville', 'Sulligent', 'Good Hope', 'Tallassee', 'Coffeeville', 'Addison', 'Muscadine', 'Crossville', 'Childersburg', 'Hurtsboro', 'Bremen', 'Rosinton', 'Louisville', 'Cowarts', 'Oakman', 'Jack', 'Whistler', 'Brewton', 'Clayton', 'Dolomite', 'Satsuma', 'Demopolis', 'Guin', 'Elsanor', 'Vina', 'Thorsby', 'Cottonwood', 'Opelika', 'Margaret', 'Haleyville', 'Tanner', 'Sand Rock', 'Headland', 'Summerdale', 'York', 'Laceys Spring', 'Millbrook', 'Jackson', 'Eldridge', 'Evergreen', 'Irondale', 'Axis', 'Carrollton', 'Waterloo', 'Forestdale', 'Foley', 'Ashville', 'Oxford', 'Guntersville', 'Ohatchee', 'Graysville', 'Riverside', 'Brent', 'Henderson', 'Cherokee Ridge', 'Hanceville', 'Dodge City', 'Midland City', 'Semmes', 'Argo', 'Marion', 'Oneonta', 'Sterrett', 'Trussville', 'Randolph', 'Muscle Shoals', 'Hueytown', 'Phenix City', 'Munford',

In [6]:
import json


with open('data_dict.json', 'w') as f:
    json.dump(data_dict, f)


In [7]:
from flask import Flask, request, render_template
from xgboost import XGBRegressor
import numpy as np
import joblib
import json

app = Flask(__name__)

# Carga el modelo previamente entrenado
model = joblib.load('G:/Mi unidad/Data Science/proyecto/Proyecto-predictor-de-precios-de-propiedades-en-EEUU/models/xgboost_optimizado.pkl')

# Carga el diccionario de estados y ciudades desde el archivo JSON
with open('data_dict.json', 'r') as f:
    data_dict = json.load(f)

@app.route('/')
def index():
    # Pasar el diccionario de estados a ciudades a la plantilla
    return render_template('index.html', data_dict=data_dict)



@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Captura y procesa los datos del formulario
        # ...
        
        # Realiza la predicción
        prediction = model.predict([input_data])

        # Envía la predicción al usuario
        return render_template('result.html', prediction=prediction)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\rgarciamontero\AppData\Local\miniconda3\envs\entorno1\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
from flask import Flask, request, render_template
from xgboost import XGBRegressor
import numpy as np
import joblib
import json
import os

app = Flask(__name__)

# Carga el modelo previamente entrenado
model = joblib.load('G:/Mi unidad/Data Science/proyecto/Proyecto-predictor-de-precios-de-propiedades-en-EEUU/models/xgboost_optimizado.pkl')

# Verificamos el trayectorio actual y aseguramos de que estamos leyendo el archivo
print("Current directory:", os.getcwd())  # Esto ayuda a verificar dónde está corriendo el script

# Ruta relativa o absoluta al archivo JSON
json_path = 'data_dict.json'
if not os.path.exists(json_path):
    print(f"File not found: {json_path}")

try:
    with open(json_path, 'r') as f:
        data_dict = json.load(f)
except Exception as e:
    print(f"Error loading JSON: {e}")

@app.route('/')
def index():
    # Pasar el diccionario de estados a ciudades a la plantilla
    return render_template('index.html', data_dict=data_dict)

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Captura y procesa los datos del formulario
        # ...

        # Realiza la predicción
        prediction = model.predict([input_data])

        # Envía la predicción al usuario
        return render_template('result.html', prediction=prediction)

if __name__ == '__main__':
    app.run(debug=True)


Current directory: g:\Mi unidad\Data Science\proyecto\Proyecto-predictor-de-precios-de-propiedades-en-EEUU\src
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1